In [1]:
%load_ext autoreload
%autoreload 2
import sys
print(sys.executable)
import socket
print(socket.gethostname())
import os
os.chdir("/home/icb/alioguz.can/projects/campa_pt")

/home/icb/alioguz.can/miniconda3/envs/campa_pt/bin/python
gpusrv32.scidom.de


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
TF_ENABLE_ONEDNN_OPTS=0

In [3]:
from pathlib import Path
from campa.tl import (
    Cluster,
    Estimator,
    TorchEstimator,
    Predictor,
    TorchPredictor,
    Experiment,
    TorchExperiment,
    ModelComparator,
    run_experiments,
    run_torch_experiments
)
from campa.data import MPPData
from campa.utils import init_logging
from campa.constants import campa_config
# init logging with level INFO=20, WARNING=30
init_logging(level=30)
# read correct campa_config -- created with setup.ipynb
CAMPA_DIR = Path.cwd()
campa_config.config_fname = CAMPA_DIR / "notebooks/params/campa.ini"
print(campa_config)

/home/icb/alioguz.can/miniconda3/envs/campa_pt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-13 15:55:51.705401: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-13 15:56:49.625948: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-01-13 15:56:49.640837: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

Reading config from /ictstr01/home/icb/alioguz.can/projects/campa_pt/notebooks/params/campa.ini
CAMPAConfig (fname: /ictstr01/home/icb/alioguz.can/projects/campa_pt/notebooks/params/campa.ini)
EXPERIMENT_DIR: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments
BASE_DATA_DIR: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_data
CO_OCC_CHUNK_SIZE: 10000000.0
data_config/exampledata: /home/icb/alioguz.can/projects/campa_pt/notebooks/params/ExampleData_constants.py



In [4]:
# import tensorflow as tf
# from tensorflow.keras import backend as K
# import torch
# torch.cuda.empty_cache()
# # Clear the current session
# K.clear_session()

# # Optionally, reset the default graph
# tf.compat.v1.reset_default_graph()

In [5]:
# get Experiments from config
exps = Experiment.get_experiments_from_config("notebooks/params/example_experiment_params.py")

In [6]:
# run_experiments(exps, mode="trainval")

In [7]:
exp = exps[0]
print("Experiment name:", exp.name)
print("Experiment is stored in:", exp.full_path)

Experiment name: VAE
Experiment is stored in: /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE


In [8]:
est = Estimator(exps[0])

2025-01-13 15:58:46.263092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-01-13 15:58:46.353774: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2025-01-13 15:58:46.506295: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
_ = est.train_model()

4/4 [==============================] - 9s 1s/step - loss: 5397.7578 - decoder_loss: 5397.7217 - latent_loss: 0.0361 - decoder_mean_squared_error: 0.8036 - latent_kl_loss: 0.0355 - val_loss: 5319.9741 - val_decoder_loss: 5319.9424 - val_latent_loss: 0.0319 - val_decoder_mean_squared_error: 0.7455 - val_latent_kl_loss: 0.0319


In [28]:
est.model.model.loss

{'decoder': <tensorflow.python.eager.polymorphic_function.polymorphic_function.Function at 0x7f9a7e3a8130>,
 'latent': <tensorflow.python.eager.polymorphic_function.polymorphic_function.Function at 0x7f9a7ed72970>}

In [18]:
print(est.model.model.summary())

Model: "VAEModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 3, 34)]   0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 3, 3, 32)     1120        ['input_1[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 288)          0           ['conv2d[0][0]']                 
                                                                                                  
 dense (Dense)                  (None, 32)           9248        ['flatten[0][0]']                
                                                                                           

In [19]:
torch_exps = TorchExperiment.get_experiments_from_config("notebooks/params/example_experiment_params_torch.py")

In [13]:
est_torch = TorchEstimator(torch_exps[0])
print(est_torch.model)
est_torch.model.total_trainable_params()

VAEModelTorch(
  (encoder_input): ModuleList()
  (decoder_input): ModuleList()
  (encoder): Sequential(
    (0): Conv2d(34, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=288, out_features=32, bias=True)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=16, bias=True)
    (6): ReLU()
    (7): Linear(in_features=16, out_features=32, bias=True)
  )
  (latent): Linear(in_features=16, out_features=32, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=34, bias=True)
  )
)
12018


In [14]:
est_torch.config["training"]

{'learning_rate': 0.001,
 'epochs': 1,
 'batch_size': 128,
 'loss': {'decoder': <function campa.tl._losses.sigma_vae_mse_torch(y_true, y_pred)>,
  'latent': <function campa.tl._losses.kl_loss_torch(y_true, y_pred)>},
 'loss_weights': {'decoder': 1, 'latent': 1},
 'loss_warmup_to_epoch': {},
 'metrics': {'decoder': MSELoss(),
  'latent': <function campa.tl._losses.kl_loss_torch(y_true, y_pred)>},
 'save_model_weights': True,
 'save_history': True,
 'overwrite_history': True}

In [ ]:
est_torch.train_model()

In [21]:
pred_torch = TorchPredictor(torch_exps[0])
pred_torch.evaluate_model()

In [22]:
results_folder = os.path.join(pred_torch.exp.full_path, f"results_epoch{pred_torch.est.epoch:03d}")
print("Results folder", results_folder)
print(os.listdir(results_folder))

Results folder /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE/results_epoch000
['val', 'val_imgs']


In [23]:
print(MPPData.from_data_dir(os.path.join(results_folder, "val"), data_config="ExampleData"))

MPPData for ExampleData (123 mpps with shape (3, 3, 34) from 8 objects). Data keys: ['obj_ids', 'x', 'y', 'mpp', 'conditions', 'labels', 'latent'].


In [24]:
cl = Cluster.from_exp_split(torch_exps[0])

Cannot read with memmap:  /home/icb/alioguz.can/projects/campa_pt/notebooks/example_experiments/test/VAE/results_epoch001/val/clustering.npy


AssertionError: 

In [ ]:
print(cl.config["leiden_resolution"])
cl.create_clustering()

In [20]:
# run_torch_experiments(torch_exps, mode="trainval")